In [2]:
!pip install lingam

In [6]:
#German
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn.model_selection import train_test_split

from csse import CSSE
from prepare_dataset import *

import pickle

import numpy as np
import pandas as pd
import json
import graphviz
import lingam
from lingam.utils import print_causal_directions, print_dagc, make_dot

from tqdm import tqdm
import random as rnd
import ast
from IPython.display import display
import time
import warnings

warnings.filterwarnings('ignore')

In [6]:
#German
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn.model_selection import train_test_split

from csse import CSSE
from prepare_dataset import *

import pickle

import numpy as np
import pandas as pd
import graphviz
import lingam
from lingam.utils import print_causal_directions, print_dagc, make_dot

from tqdm import tqdm
import random as rnd

from IPython.display import display

import warnings

warnings.filterwarnings('ignore')



class CCSSE:
    def __init__(self, dataset, samples = None, K = 5, generation = 10, bb_model):
        self.df_datasets = self.load_df_dataset()
        self.dataset = dataset
        self.samples = samples
        self.K = K
        self.generation = generation
        
#         self.x_train, self.x_test, self.y_train, self.y_test, self.dfx_full, self.dfy_full = self.get_datasets_train_test()
        self.x_train, self.x_test, self.y_train, self.y_test, self.dfx_full, self.dfy_full = self.get_dataset()

        self.bb_model, self.p = self.get_bb_model(bb_model)
        self.explainerCSSE = self.get_model_contrafactual()
        self.model_causal, self.df_causal_effects, self.df_error, self.causal_order = self.get_model_causality()
    
        self.run_dict = {}
        self.run_non_causal_dict = {}
        
    def load_df_dataset(self):
        def convert_to_list(val):
            try:
                return ast.literal_eval(val) if isinstance(val, str) and val.startswith('[') and val.endswith(']') else val
            except (ValueError, SyntaxError):
                return val
            
        df = pd.read_parquet("datasets/df_map_inference_datasets.parquet")
        df['path'] = df['path'].apply(convert_to_list)
        
        return df


    def get_dataset(self):
        dataset_dict = self.df_datasets[self.df_datasets['name'] == self.dataset].iloc[0].to_dict()
        
        if isinstance(dataset_dict['path'], list):
            if 'Column' in dataset_dict['classe']:
                df_train = pd.read_csv(f"s3://omar-testes-gerais/artigos/artifacts/datasets/{dataset_dict['path'][0]}", header = None)
                df_test = pd.read_csv(f"s3://omar-testes-gerais/artigos/artifacts/datasets/{dataset_dict['path'][1]}", header = None)
                class_name = int(dataset_dict['class'].split('Column')[1])
            else:
                df_train = pd.read_csv(f"s3://omar-testes-gerais/artigos/artifacts/datasets/{dataset_dict['path'][0]}")
                df_test = pd.read_csv(f"s3://omar-testes-gerais/artigos/artifacts/datasets/{dataset_dict['path'][1]}")
                class_name = dataset_dict['class']
            
            x_train = df_train.drop(columns=[class_column])
            y_train = df_train[class_column]

            # Dividindo o df_test
            x_test = df_test.drop(columns=[class_column])
            y_test = df_test[class_column]
            
            dfx_full = pd.concat([x_train, x_test])
            dfy_full = pd.concat([y_train, y_test])
            
        
        else:
            if 'Column' in dataset_dict['classe']:
                df_main = pd.read_csv(f"s3://omar-testes-gerais/artigos/artifacts/datasets/{dataset_dict['path']}", header = None)
                class_name = int(dataset_dict['classe'].split('Column')[1])
            else:
                df_main = pd.read_csv(f"s3://omar-testes-gerais/artigos/artifacts/datasets/{dataset_dict['path']}")
                class_name = dataset_dict['classe']
                
            columns = df_main.columns
            columns_tmp = list(columns)
            columns_tmp.remove(class_name)

            x_train, x_test, y_train, y_test = train_test_split(df_main[columns_tmp], df_main[class_name], test_size=0.1)

            dfx_full = pd.concat([x_train, x_test])
            dfy_full = pd.concat([y_train, y_test])
            
        return x_train, x_test, y_train, y_test, dfx_full, dfy_full
    
    def get_datasets_train_test(self):
        if self.dataset == "german_short":
            map_columns = {
                'Unnamed: 0': 'index',
                'x0': 'Sex',
                'x1': 'Age',
                'x2': 'Credit',
                'x3': 'LoanDuration',
            }
            x_train = pd.read_csv("data/algrec_german/X_train_short.csv").rename(columns = map_columns)
            x_test = pd.read_csv("data/algrec_german/X_test_short.csv").rename(columns = map_columns)
            y_train = pd.read_csv("data/algrec_german/y_train_short.csv").rename(columns={'Unnamed: 0': 'index'})
            y_test = pd.read_csv("data/algrec_german/y_test_short.csv").rename(columns={'Unnamed: 0': 'index'})
            x_train = x_train.set_index('index')
            x_test = x_test.set_index('index')
            y_train = y_train.set_index('index')
            y_test = y_test.set_index('index')
            dfx_full = pd.concat([x_train, x_test])
            dfy_full = pd.concat([y_train, y_test])

        elif self.dataset == "german_medium":
            map_columns = {
                'Unnamed: 0': 'index',
                'x0': 'Sex',
                'x1': 'Age',
                'x2': 'Credit',
                'x3': 'LoanDuration',
                'x4': 'CheckingAccountBalance',
                'x5':'SavingsAccountBalance',
                'x6':'HousingStatus'
            }
            x_train = pd.read_csv("data/algrec_german/X_train.csv").rename(columns = map_columns)
            x_test = pd.read_csv("data/algrec_german/X_test.csv").rename(columns = map_columns)
            y_train = pd.read_csv("data/algrec_german/y_train.csv").rename(columns={'Unnamed: 0': 'index'})
            y_test = pd.read_csv("data/algrec_german/y_test.csv").rename(columns={'Unnamed: 0': 'index'})
            x_train = x_train.set_index('index')
            x_test = x_test.set_index('index')
            y_train = y_train.set_index('index')
            y_test = y_test.set_index('index')
            dfx_full = pd.concat([x_train, x_test])
            dfy_full = pd.concat([y_train, y_test])

        elif self.dataset == "german_full":
            df_main = prepare_german_dataset("german_credit.csv", "data/")
            columns = df_main.columns
            class_name = 'default' # default = 0 = "Good class" / default = 1 = "Bad class" 
            columns_tmp = list(columns)
            columns_tmp.remove(class_name)

            x_train, x_test, y_train, y_test = train_test_split(df_main[columns_tmp], df_main[class_name], test_size=0.1)

            dfx_full = pd.concat([x_train, x_test])
            dfy_full = pd.concat([y_train, y_test])
            
        elif self.dataset == 'np':
            df_main = pd.read_csv("data/breast_coimbra_np.csv")
            columns = df_main.columns
            class_name = 'Classification' # default = 0 = "Good class" / default = 1 = "Bad class" 
            columns_tmp = list(columns)
            columns_tmp.remove(class_name)

            x_train, x_test, y_train, y_test = train_test_split(df_main[columns_tmp], df_main[class_name], test_size=0.1)

            dfx_full = pd.concat([x_train, x_test])
            dfy_full = pd.concat([y_train, y_test])
            
        elif self.dataset == 'nm':
            df_main = pd.read_csv("data/heloc_dataset_v1_nm.csv")
            columns = df_main.columns
            class_name = 'RiskPerformance' # default = 0 = "Good class" / default = 1 = "Bad class" 
            columns_tmp = list(columns)
            columns_tmp.remove(class_name)

            x_train, x_test, y_train, y_test = train_test_split(df_main[columns_tmp], df_main[class_name], test_size=0.1)

            dfx_full = pd.concat([x_train, x_test])
            dfy_full = pd.concat([y_train, y_test])
            
        elif self.dataset == 'cm':
            df_main = pd.read_csv("data/house_votes_84_processada_cm.csv")
            columns = df_main.columns
            class_name = 'Class Name' # default = 0 = "Good class" / default = 1 = "Bad class" 
            columns_tmp = list(columns)
            columns_tmp.remove(class_name)

            x_train, x_test, y_train, y_test = train_test_split(df_main[columns_tmp], df_main[class_name], test_size=0.1)

            dfx_full = pd.concat([x_train, x_test])
            dfy_full = pd.concat([y_train, y_test])
            
        elif self.dataset == 'ng':
            df_main = pd.read_csv("data/ionosphere_ng.csv")
            columns = df_main.columns
            class_name = 'target' # default = 0 = "Good class" / default = 1 = "Bad class" 
            columns_tmp = list(columns)
            columns_tmp.remove(class_name)

            x_train, x_test, y_train, y_test = train_test_split(df_main[columns_tmp], df_main[class_name], test_size=0.1)

            dfx_full = pd.concat([x_train, x_test])
            dfy_full = pd.concat([y_train, y_test])
            
        elif self.dataset == 'tokyo_ng':
            df_main = pd.read_csv("data/Tokyo_ng.csv").rename(columns = {'Unnamed: 44': 'class'})
            columns = df_main.columns
            class_name = 'class' # default = 0 = "Good class" / default = 1 = "Bad class" 
            columns_tmp = list(columns)
            columns_tmp.remove(class_name)

            x_train, x_test, y_train, y_test = train_test_split(df_main[columns_tmp], df_main[class_name], test_size=0.1)

            dfx_full = pd.concat([x_train, x_test])
            dfy_full = pd.concat([y_train, y_test])
            
        elif self.dataset == 'breast-cancer_ng':
            df_main = pd.read_csv("data/breast-cancer_ng.csv")
            columns = df_main.columns
            class_name = 'diagnosis' # default = 0 = "Good class" / default = 1 = "Bad class" 
            columns_tmp = list(columns)
            columns_tmp.remove(class_name)

            x_train, x_test, y_train, y_test = train_test_split(df_main[columns_tmp], df_main[class_name], test_size=0.1)

            dfx_full = pd.concat([x_train, x_test])
            dfy_full = pd.concat([y_train, y_test])
        
        elif self.dataset == 'cp':
            df_train = pd.read_csv("data/monks-1_train_cp.csv")

            #Get the input features
            columns = df_train.columns
            class_name = 'Class' # default = 0 = "Good class" / default = 1 = "Bad class" 
            columns_tmp = list(columns)
            columns_tmp.remove(class_name)
            columns_tmp.remove('Id')

            x_train = df_train[columns_tmp]
            y_train = df_train[['Class']]

            # x_train, x_test, y_train, y_test = train_test_split(df_main[columns_tmp], df_main[class_name], test_size=0.1)

            model = RandomForestClassifier()  
            model.fit(x_train, y_train)

            df_test = pd.read_csv('data/monks-1_test_cp.csv')

            #Get the input features
            columns = df_test.columns
            class_name = 'Class' # default = 0 = "Good class" / default = 1 = "Bad class" 
            columns_tmp = list(columns)
            columns_tmp.remove(class_name)
            columns_tmp.remove('Id')

            x_test = df_test[columns_tmp]
            y_test = df_test[['Class']]

            dfx_full = pd.concat([x_train, x_test])
            dfy_full = pd.concat([y_train, y_test])
            
        elif self.dataset == 'cg':
            df_main = pd.read_csv("data/mushrooms_processada_cg.csv")[:1000]
            columns = df_main.columns
            class_name = 'class' # default = 0 = "Good class" / default = 1 = "Bad class" 
            columns_tmp = list(columns)
            columns_tmp.remove(class_name)

            x_train, x_test, y_train, y_test = train_test_split(df_main[columns_tmp], df_main[class_name], test_size=0.1)

            dfx_full = pd.concat([x_train, x_test])
            dfy_full = pd.concat([y_train, y_test])
            
        elif self.dataset == 'Phishing_cg':
            df_main = pd.read_csv("data/Phishing_cg.csv")[2:1000].astype(float)
            columns = df_main.columns
            class_name = 'Class' # default = 0 = "Good class" / default = 1 = "Bad class" 
            columns_tmp = list(columns)
            columns_tmp.remove(class_name)

            x_train, x_test, y_train, y_test = train_test_split(df_main[columns_tmp], df_main[class_name], test_size=0.1)

            dfx_full = pd.concat([x_train, x_test])
            dfy_full = pd.concat([y_train, y_test])

        else:
            x_train = pd.DataFrame()
            x_test = pd.DataFrame()
            y_train = pd.DataFrame()
            y_test = pd.DataFrame()
            dfx_full = pd.DataFrame()
            dfy_full = pd.DataFrame()

        return x_train, x_test, y_train, y_test, dfx_full, dfy_full
    

    def get_bb_model(self, bb_model_name):
        
        if bb_model_name == 'rf':
            bb_model = RandomForestClassifier()  
            bb_model.fit(self.x_train, self.y_train)

            p = bb_model.predict(self.x_test)

            print(classification_report(self.y_test, p))

            return bb_model, p
        elif bb_model_name == 'rn':
            pass

    def get_model_contrafactual(self):
        return CSSE(self.dfx_full, self.bb_model, K = self.K, num_gen = self.generation)

    def get_model_causality(self):
        model_causal = lingam.DirectLiNGAM()
        model_causal.fit(self.dfx_full)
            
        labels = [f'{i}' for i in self.dfx_full.columns]
        causal_order = [labels[x] for x in model_causal.causal_order_]
        
        matrix = model_causal.adjacency_matrix_
        from_list = []
        to_list = []
        effect_list = []

        # Iteração sobre a matriz para extrair os valores e suas posições
        for i in range(len(matrix)):
            for j in range(len(matrix[i])):
                if matrix[i][j] != 0:
                    from_list.append(j)
                    to_list.append(i)
                    effect_list.append(matrix[i][j])

        # Criando o DataFrame
        df_causal_effects = pd.DataFrame({'from': from_list, 'to': to_list, 'effect': effect_list})
        labels = [f'{i}' for i in self.dfx_full.columns]
        df_causal_effects['from'] = df_causal_effects['from'].apply(lambda x : labels[x])
        df_causal_effects['to'] = df_causal_effects['to'].apply(lambda x : labels[x])


        matrix_error = model_causal.get_error_independence_p_values(self.dfx_full)
        from_list = []
        to_list = []
        effect_list = []

        # Iteração sobre a matriz para extrair os valores e suas posições
        for i in range(len(matrix_error)):
            for j in range(i + 1, len(matrix_error[i])):
                if matrix_error[i][j] != 0:
                    from_list.append(j)
                    to_list.append(i)
                    effect_list.append(matrix_error[i][j])

        # Criando o DataFrame
        df_error = pd.DataFrame({'from': from_list, 'to': to_list, 'effect': effect_list})
        labels = [f'{i}' for i in self.dfx_full.columns]
        df_error['from'] = df_error['from'].apply(lambda x : labels[x])
        df_error['to'] = df_error['to'].apply(lambda x : labels[x])
        

        return model_causal, df_causal_effects, df_error, causal_order
        
    
    def print_causal_graph(self):
        make_dot(self.model_causal.adjacency_matrix_)

    def run_non_causal(self):
        self.run_non_causal_dict = {}

        if isinstance(self.samples, list):
            self.create_run_dict(self)
            for sample in self.samples:
                self.run_non_causal_sample(sample)
                
        elif isinstance(self.samples, int):
            for sample in range(self.samples):
                self.run_non_causal_sample(sample)
        
        else: 
            for sample in range(10):
                self.run_non_causal_sample(sample)
                
    def run_non_causal_sample(self, sample):
        self.run_non_causal_dict[sample] = {}
        original_instance = self.x_test.iloc[sample].copy()
        contrafactual_set, solution = self.explainerCSSE.explain(original_instance, self.p[sample]) #Method returns the list of counterfactuals and the explanations generated from them

        self.run_non_causal_dict[sample]['solution'] = solution

    def run_causal(self):
        start_time = time.time()
        self.run_dict = {}
        self.run_dict['global_numbers'] = {
                    "global_quant_changes": 0,
                    "global_quant_causal_changes": 0,
                    "global_quant_causal_rules": 0,
                    "global_quant_zeros_causal": 0,
                    "global_quant_full_causal": 0,
                    "global_quant_causal_contrafac": 0,
                    "global_quant_maioria_causal_satisfeita": 0,
                    "global_quant_contrafac_unico": 0,
            }
        self.global_quant_contrafac_max = 0
        if isinstance(self.samples, list):
            for sample in self.samples:
                self.run_causal_sample(sample)
                
        elif isinstance(self.samples, int):
            for sample in range(self.samples):
                self.run_causal_sample(sample)
        
        else: 
            for sample in range(10):
                self.run_causal_sample(sample)
        
        self.global_quant_contrafac_max = self.K * len(self.run_dict)
        self.run_dict['global_numbers']['global_timing_run_causal'] = time.time() - start_time


    def run_causal_sample(self, sample):
        if isinstance(self.samples, list):
            original_instance = self.dfx_full.iloc[sample]
        else:
            original_instance = self.x_test.iloc[sample]
        self.run_dict[sample] = {}
        self.run_dict[sample]['original_instance'] = original_instance

        print(f'Running original instance:\n {display(original_instance)}')

        causal_explain = self.get_causal_explain(sample)
        self.run_dict[sample]['causal_explain'] = causal_explain

        list_analyse = []
        for contrafactual in causal_explain[0]:
            list_analyse.append(self.analyse_contrafac(contrafactual, causal_explain[1], causal_explain[2]))

        self.run_dict[sample]['list_analyse'] = list_analyse
        self.analyse_explaination(sample)

    def analyse_contrafac(self, contrafac, df, original_ind):
        columns = [x.column for x in contrafac]
        condicao = (df['to'].isin(columns)) & (df['from'].isin(columns))
        ind = original_ind[columns]
        return [contrafac, df[condicao], ind]

    def get_causal_explain(self, sample):
        if isinstance(self.samples, list):
            original_ind = self.dfx_full.iloc[sample].copy()
        else:
            original_ind = self.x_test.iloc[sample].copy() #Original instance
        #self.ind_cur_class = ind_cur_class #Index in the shap corresponds to the original instance class
        self.explainerCSSE.current_class = self.p[sample] #Original instance class
        self.explainerCSSE.original_ind = original_ind
        
        ind_cur_class = self.explainerCSSE.getBadClass()

        #Gets the valid values range of each feature
        features_range = []
        features_range = self.explainerCSSE.getFeaturesRange()

        #The DataFrame df will have the current population
        df = pd.DataFrame(columns=self.explainerCSSE.input_dataset.columns)

        #Generates the initial population with popinitial mutants        
        self.explainerCSSE.getPopInicial(df, features_range)
        df_causal = df.copy()
        dict_dfs = {}

        # for g in tqdm(range(self.explainerCSSE.num_gen), desc= "Processing..."):
        for g in tqdm(range(self.generation), desc= "Processing..."):

            #To use on the parents of each generation
            old_parents = pd.DataFrame(columns=self.explainerCSSE.input_dataset.columns)

            #Copy parents to the next generation
            old_parents = df_causal.copy()
            dict_dfs[g] = {}

            parents_causal = self.apply_causality(old_parents)
            dict_dfs[g]['causal_parents'] = parents_causal
            #df will contain the new population
            df_causal = pd.DataFrame(columns=self.explainerCSSE.input_dataset.columns)
            evaluation_causal = []

            #Assessing generation counterfactuals
            self.explainerCSSE.fitness(dict_dfs[g]['causal_parents'], evaluation_causal, ind_cur_class)

            #The original individual will always be in the 0 position of the df - So that it is normalized too (it will be used later in the distance function)
            df_causal.loc[0] = original_ind.copy()

            #Copies to the next generation the per_elit best individuals
            self.explainerCSSE.elitism(evaluation_causal, df_causal, parents_causal)
            number_cross_repetitions = 0
            while len(df_causal) < self.explainerCSSE.pop_size + 1: #+1, as the 1st position is used to store the reference individual
                number_cross_repetitions_causal = self.explainerCSSE.crossover(df_causal, parents_causal, evaluation_causal, number_cross_repetitions)

                mutation_op = rnd.random()
                if mutation_op <= self.explainerCSSE.mutation_proba:
                    self.explainerCSSE.mutation(df_causal, len(df_causal) - 1, features_range)


        evaluation = []
        evaluation_causal = []

        #Evaluating the latest generation
        self.explainerCSSE.fitness(df_causal, evaluation_causal, ind_cur_class)

        #Order the last generation by distance to the original instance     
        evaluation_causal.sort(key=lambda individual: individual.aval_norm) 

        #Getting the counterfactual CAUSAL set
        contrafactual_set_causal, solution_list_causal = self.explainerCSSE.getContrafactual(df_causal, evaluation_causal) 

        dict_dfs['contrafactual_set_causal'] = contrafactual_set_causal
        dict_dfs['solution_list_causal'] = solution_list_causal
        
        df_contrafac_causal = self.get_contrafac_df_causal(solution_list_causal)
        return [solution_list_causal, df_contrafac_causal, original_ind]
    

    def apply_causality(self, df):
        df_apply_causal = pd.DataFrame(columns = df.columns)
        original = df.iloc[0]
        df_apply_causal.loc[0] = original
        for index, df_row in df.iloc[1:].iterrows():
            causal_ind = df_row.copy()
            for column in self.causal_order:
                value_diff = causal_ind[column] - original[column]
                if value_diff != 0:
                    tmp_effects = self.df_causal_effects[self.df_causal_effects['from'] == column]
                    for index, row in tmp_effects.iterrows():
    #                     prob = rnd.random()
    #                     if row['probability'] <= prob:
                        tmp_error = self.df_error[self.df_error['from'].isin([column, row['to']]) | self.df_error['to'].isin([column, row['to']])]
                        error_value = tmp_error['effect'].iloc[0]
    #                     print(f'error value = {error_value}')
                        causal_ind[row['to']] = causal_ind[row['to']] + (value_diff * row['effect']) + tmp_error['effect'].iloc[0]
            df_apply_causal.loc[len(df_apply_causal)] = causal_ind
        return df_apply_causal


    def get_contrafac_df_causal(self, solution_list_causal):
        lista_solution_causal = [[t.column for t in sublist] for sublist in solution_list_causal]

        # Inicializa uma lista para armazenar os resultados
        resultados = []

        # Loop sobre os valores na lista
        for lista_valores in lista_solution_causal:
            if len(lista_valores) > 1:
                for v1 in lista_valores:
                    for v2 in lista_valores:
                        if v1 != v2:
                            # Cria uma condição para cada par de valores diferentes na lista
                            condicao = (self.df_causal_effects['to'].isin([v1, v2])) & (self.df_causal_effects['from'].isin([v1, v2]))
                            # Realiza a busca no DataFrame usando a condição e armazena os resultados
                            resultados.append(self.df_causal_effects[condicao])

        # Concatena os resultados em um único DataFrame
        if resultados:
            resultado_final = pd.concat(resultados)
            resultado_final = resultado_final.drop_duplicates()
        else:
            resultado_final = pd.DataFrame(columns = self.df_causal_effects.columns)
            
        return resultado_final
    

    def analyse_explaination(self, sample):
        self.run_dict[sample]['data_analysis'] = []
        for i, content in enumerate(self.run_dict[sample]['list_analyse']):
            self.global_quant_contrafac_max += 1
            controle = {}
            causal = content[0]
            df = content[1]
            ori = content[2]
            
            
            num_changes = len(causal)
            self.run_dict['global_numbers']['global_quant_changes'] += num_changes
            
            num_causal_rules = len(df)
            self.run_dict['global_numbers']['global_quant_causal_rules'] += num_causal_rules
            
            for attr in causal:
                key = attr.column
                if attr.value > ori[key]:
                    controle[key] = 'mais'
                else:
                    controle[key] = 'menos'

            df_temp = df.copy()
            df_temp['from'] = df['from'].map(controle)
            df_temp['to'] = df['to'].map(controle)
            df_temp['causal'] = df_temp.apply(self.verificar_condicoes, axis = 1)
            data_dict = {}

            data_dict['df_respeita_causal'] = df_temp
            data_dict['contrafactual_causal'] = causal
            data_dict['df_causal_effects'] = df
            
            self.run_dict[sample]['data_analysis'].append(data_dict)


            causal_finds = df_temp['causal'].sum()
            self.run_dict['global_numbers']['global_quant_causal_changes'] += causal_finds
            
            # print(f'causal = \n{causal}\n')
            # print(f'original = \n{ori}\n')
            # print(f'df_temp = \n{display(df_temp)}\n')
            
            if len(df_temp) > 0:
                if causal_finds > 0:
                    self.run_dict['global_numbers']['global_quant_causal_contrafac'] += 1
                else:
                    # print(f'nenhuma relaçao causal satisfeita')
                    self.run_dict['global_numbers']['global_quant_zeros_causal'] += 1
    #                 display(df_temp)
    #                 print(f"original = {ori}")
    #                 print(f"causal = {causal}")

                if causal_finds == num_causal_rules:
                    self.run_dict['global_numbers']['global_quant_full_causal'] += 1
                    # if causal_finds > 2:
                        # print(f'todas > 2 relaçoes causais satisfeitas')
    #                     display(df_temp)
    #                     print(f"original = {ori}")
    #                     print(f"causal = {causal}")
                    # elif causal_finds == 1:
                        # print(f'todas = 1 relaçoes causais satisfeitas')
                
                if causal_finds >= (len(df_temp)/2):
                    self.run_dict['global_numbers']['global_quant_maioria_causal_satisfeita'] += 1
            else:
    #             if len(causal) > 0:
                self.run_dict['global_numbers']['global_quant_contrafac_unico'] += 1
        
    def verificar_condicoes(self, row):
        if (row['from'] == 'mais' and row['to'] == 'mais' and row['effect'] > 0):
            return True
        elif row['from'] == 'menos' and row['to'] == 'menos' and row['effect'] > 0:
            return True
        elif row['from'] == 'mais' and row['to'] == 'menos' and row['effect'] < 0:
            return True
        elif row['from'] == 'menos' and row['to'] == 'mais' and row['effect'] < 0:
            return True
        else:
            return False
    
            

    def show_metrics(self, get_output = False):
        
        print(f"Quantidade de instâncias contrafactuais = {self.global_quant_contrafac_max}")
        print(f'Quantidade de relações causais na base de dados = {len(self.df_causal_effects)}')
        print(f"Quantidade de atributos modificados = {self.run_dict['global_numbers']['global_quant_changes']}")
        print(f"Quantidade de instâncias contrafactuais causais = {self.run_dict['global_numbers']['global_quant_contrafac_unico'] + self.run_dict['global_numbers']['global_quant_causal_contrafac']}")
        print(f"Quantidade de relações causais analisadas = {self.run_dict['global_numbers']['global_quant_causal_rules']}")
        print(f"Quantidade de relações causais satisfeitas = {self.run_dict['global_numbers']['global_quant_causal_changes']}")
        print(f"Quantidade de instâncias contrafactuais com um único atributo modificado = {self.run_dict['global_numbers']['global_quant_contrafac_unico']}")
        print(f"Tempo de execução = {self.run_dict['global_numbers']['global_timing_run_causal']}")
        
        if get_output:
            metrics_dict = {
                "Quantidade de instâncias contrafactuais": self.global_quant_contrafac_max,
                "Quantidade de relações causais na base de dados": len(self.df_causal_effects),
                "Quantidade de atributos modificados": self.run_dict['global_numbers']['global_quant_changes'],
                "Quantidade de instâncias contrafactuais causais": self.run_dict['global_numbers']['global_quant_contrafac_unico'] + self.run_dict['global_numbers']['global_quant_causal_contrafac'],
                "Quantidade de relações causais analisadas": self.run_dict['global_numbers']['global_quant_causal_rules'],
                "Quantidade de relações causais satisfeitas": self.run_dict['global_numbers']['global_quant_causal_changes'],
                "Quantidade de instâncias contrafactuais com um único atributo modificado": self.run_dict['global_numbers']['global_quant_contrafac_unico'],
                "Tempo de execução": self.run_dict['global_numbers']['global_timing_run_causal']
            }
        
            return metrics_dict
              
              
              


In [7]:
df_map_inference_datasets = pd.read_parquet("datasets/df_map_inference_datasets.parquet")
df_map_inference_datasets.columns

Index(['name', 'path', 'classe'], dtype='object')

In [ ]:
def get_causal_metrics(row):
    try:
        print(row)
        ccsse = CCSSE(row['name'], samples = 10, K = 10, generation= 10)
        ccsse.run_causal()
        return ccsse.show_metrics(get_output = True)
    except Exception as e:
        return e

df_map_inference_datasets['causal_metrics'] = df_map_inference_datasets.apply(get_causal_metrics, axis = 1)

name                           Adult
path      adult/adult_processada.csv
classe               Above/Below 50K
Name: 0, dtype: object
              precision    recall  f1-score   support

       <=50K       0.88      0.92      0.90      3741
        >50K       0.70      0.60      0.65      1144

    accuracy                           0.85      4885
   macro avg       0.79      0.76      0.77      4885
weighted avg       0.84      0.85      0.84      4885

name                                   Australian
path      Australian/Credit_Card_Applications.csv
classe                                      Class
Name: 1, dtype: object
              precision    recall  f1-score   support

           0       0.89      0.98      0.93        42
           1       0.96      0.81      0.88        27

    accuracy                           0.91        69
   macro avg       0.92      0.90      0.91        69
weighted avg       0.92      0.91      0.91        69



CustomerID    15657228.00
A1                   1.00
A2                  21.42
A3                   0.75
A4                   1.00
A5                  12.00
A6                   7.00
A7                   0.75
A8                   0.00
A9                   0.00
A10                  0.00
A11                  1.00
A12                  2.00
A13                132.00
A14                  3.00
Name: 510, dtype: float64

Running original instance:
 None


Processing...: 100%|██████████| 10/10 [00:11<00:00,  1.15s/it]


CustomerID    15697460.00
A1                   1.00
A2                  22.00
A3                   0.79
A4                   2.00
A5                   9.00
A6                   4.00
A7                   0.29
A8                   0.00
A9                   1.00
A10                  1.00
A11                  0.00
A12                  2.00
A13                420.00
A14                284.00
Name: 125, dtype: float64

...

Processing...: 100%|██████████| 10/10 [01:52<00:00, 11.20s/it]


id                         842517.000000
radius_mean                    20.570000
texture_mean                   17.770000
perimeter_mean                132.900000
area_mean                    1326.000000
smoothness_mean                 0.084740
compactness_mean                0.078640
concavity_mean                  0.086900
concave points_mean             0.070170
symmetry_mean                   0.181200
fractal_dimension_mean          0.056670
radius_se                       0.543500
texture_se                      0.733900
perimeter_se                    3.398000
area_se                        74.080000
smoothness_se                   0.005225
compactness_se                  0.013080
concavity_se                    0.018600
concave points_se               0.013400
symmetry_se                     0.013890
fractal_dimension_se            0.003532
radius_worst                   24.990000
texture_worst                  23.410000
perimeter_worst               158.800000
area_worst      

Running original instance:
 None


Processing...: 100%|██████████| 10/10 [01:46<00:00, 10.63s/it]


Quantidade de instâncias contrafactuais = 110
Quantidade de relações causais na base de dados = 260
Quantidade de atributos modificados = 215
Quantidade de instâncias contrafactuais causais = 17
Quantidade de relações causais analisadas = 1034
Quantidade de relações causais satisfeitas = 596.0
Quantidade de instâncias contrafactuais com um único atributo modificado = 1
name                                                  Churn
path      Churn/WA_Fn-UseC_-Telco-Customer-Churn_process...
classe                                                Churn
Name: 6, dtype: object
name                                  Compas
path      Compas/compas-scores-two-years.csv
classe                        two_year_recid
Name: 7, dtype: object
name                          Credit default
path      Credit default/UCI_Credit_Card.csv
classe            default.payment.next.month
Name: 8, dtype: object
              precision    recall  f1-score   support

           0       0.84      0.94      0.89      2348


In [ ]:
df_map_inference_datasets.to_parquet(f"datasets/df_map_inference_datasets_causal_metrics.parquet", engine = "pyarrow")

In [67]:
df_map_inference_datasets

,name,path,classe,causal_metrics
0,Adult,adult/adult_processada.csv,Above/Below 50K,NaN
1,Australian,Australian/Credit_Card_Applications.csv,Class,NaN
2,Banknote,Banknote/BankNote_Authentication.csv,class,NaN
3,Biodeg,Biodeg/qsar-biodeg.csv,Class,NaN
4,Breast Cancer Coimbra,Breast Cancer Coimbra/breast_coimbra.csv,Classification,NaN
5,Breast Cancer Wisconsin,Breast Cancer Wisconsin/breast-cancer.csv,diagnosis,NaN
6,Churn,Churn/WA_Fn-UseC_-Telco-Customer-Churn_process...,Churn,NaN
7,Compas,Compas/compas-scores-two-years.csv,two_year_recid,NaN
8,Credit default,Credit default/UCI_Credit_Card.csv,default.payment.next.month,NaN
9,Diabetes,Diabetes/diabetes.csv,Outcome,NaN


In [57]:
ccsse = CCSSE('Australian', samples = 10, K = 1, generation= 10)

              precision    recall  f1-score   support

           0       0.97      0.93      0.95        42
           1       0.90      0.96      0.93        27

    accuracy                           0.94        69
   macro avg       0.94      0.95      0.94        69
weighted avg       0.94      0.94      0.94        69



In [58]:
ccsse.run_causal()

CustomerID    1.558727e+07
A1            1.000000e+00
A2            2.858000e+01
A3            3.625000e+00
A4            2.000000e+00
A5            6.000000e+00
A6            4.000000e+00
A7            2.500000e-01
A8            0.000000e+00
A9            0.000000e+00
A10           0.000000e+00
A11           1.000000e+00
A12           2.000000e+00
A13           1.000000e+02
A14           1.000000e+00
Name: 269, dtype: float64

Running original instance:
 None


Processing...: 100%|██████████| 10/10 [00:11<00:00,  1.20s/it]


CustomerID    15679394.0
A1                   0.0
A2                  36.0
A3                   1.0
A4                   2.0
A5                   8.0
A6                   4.0
A7                   2.0
A8                   1.0
A9                   1.0
A10                 11.0
A11                  0.0
A12                  2.0
A13                  0.0
A14                457.0
Name: 336, dtype: float64

Running original instance:
 None


...

Running original instance:
 None


Processing...: 100%|██████████| 10/10 [00:14<00:00,  1.41s/it]


CustomerID    15723884.00
A1                   1.00
A2                  36.25
A3                   5.00
A4                   2.00
A5                   8.00
A6                   5.00
A7                   2.50
A8                   1.00
A9                   1.00
A10                  6.00
A11                  0.00
A12                  2.00
A13                  0.00
A14                368.00
Name: 93, dtype: float64

Running original instance:
 None


Processing...: 100%|██████████| 10/10 [00:13<00:00,  1.36s/it]


In [59]:
ccsse.show_metrics()

Quantidade de instâncias contrafactuais = 11
Quantidade de relações causais na base de dados = 25
Quantidade de atributos modificados = 18
Quantidade de instâncias contrafactuais causais = 10
Quantidade de relações causais analisadas = 5
Quantidade de relações causais satisfeitas = 5.0
Quantidade de instâncias contrafactuais com um único atributo modificado = 8
---------------------------------------------------------------------------------------


In [60]:
ccsse.run_dict

{'global_numbers': {'global_quant_changes': 18,
  'global_quant_causal_changes': 5.0,
  'global_quant_causal_rules': 5,
  'global_quant_zeros_causal': 0,
  'global_quant_full_causal': 2,
  'global_quant_causal_contrafac': 2,
  'global_quant_maioria_causal_satisfeita': 2,
  'global_quant_contrafac_unico': 8},
 0: {'original_instance': CustomerID    1.558727e+07
  A1            1.000000e+00
  A2            2.858000e+01
  A3            3.625000e+00
  A4            2.000000e+00
  A5            6.000000e+00
  A6            4.000000e+00
  A7            2.500000e-01
  A8            0.000000e+00
  A9            0.000000e+00
  A10           0.000000e+00
  A11           1.000000e+00
  A12           2.000000e+00
  A13           1.000000e+02
  A14           1.000000e+00
  Name: 269, dtype: float64,
  'causal_explain': [[[('A8', 1.0), ('A13', 36.75777104155044)]],
   Empty DataFrame
   Columns: [from, to, effect]
   Index: [],
   CustomerID    1.558727e+07
   A1            1.000000e+00
   A2       

In [2]:
import pandas as pd

In [13]:
def convert_to_list(val):
            try:
                return ast.literal_eval(val) if isinstance(val, str) and val.startswith('[') and val.endswith(']') else val
            except (ValueError, SyntaxError):
                return val
            
df = pd.read_parquet("datasets/df_map_inference_datasets.parquet")
df['path'] = df['path'].apply(convert_to_list)

In [7]:
df_map_inference_datasets.to_parquet(f"s3://omar-testes-gerais/artigos/artifacts/df_map_inference_datasets.parquet", engine = "pyarrow")

In [4]:
df_dataset = df_map_inference_datasets[df_map_inference_datasets['name'].isin(['Adult', 'Australian'])]
df_dataset

,name,path,classe
0,Adult,adult/adult_processada.csv,Above/Below 50K
1,Australian,Australian/Credit_Card_Applications.csv,Class


In [15]:
import boto3
s3 = boto3.client('s3')
def move_to_s3(local_path):
    if isinstance(local_path, list):
        print(local_path)
        for path in local_path:
            s3.upload_file(f"datasets/drive_raw/{path}", 'omar-testes-gerais', f"artigos/artifacts/datasets/{path}")
            print(f'{path} moved')
    else:
        s3.upload_file(f"datasets/drive_raw/{local_path}", 'omar-testes-gerais', f"artigos/artifacts/datasets/{local_path}")
        print(f'{local_path} moved')
    
df['path'].apply(move_to_s3)

adult/adult_processada.csv moved
Australian/Credit_Card_Applications.csv moved
Banknote/BankNote_Authentication.csv moved
Biodeg/qsar-biodeg.csv moved
Breast Cancer Coimbra/breast_coimbra.csv moved
Breast Cancer Wisconsin/breast-cancer.csv moved
Churn/WA_Fn-UseC_-Telco-Customer-Churn_processada.csv moved
Compas/compas-scores-two-years.csv moved
Credit default/UCI_Credit_Card.csv moved
Diabetes/diabetes.csv moved
['Driving Behavior/train_motion_data.csv', 'Driving Behavior/test_motion_data.csv']
Driving Behavior/train_motion_data.csv moved
Driving Behavior/test_motion_data.csv moved
EEg/EEG Eye State.csv moved
German/german_credit.csv moved
GiveMeSomeCredit/GiveMeSomeCredit_processada.csv moved
Heart/heart.csv moved
Heart/heart2.csv moved
Heart Failure Prediction/heart_failure_clinical_records_dataset.csv moved
HELOC/heloc_dataset_v1.csv moved
['Hill/Hill_Valley_with_noise_Training.csv', 'Hill/Hill_Valley_with_noise_Testing.csv']
Hill/Hill_Valley_with_noise_Training.csv moved
Hill/Hill_

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
19    None
20    None
21    None
22    None
23    None
24    None
25    None
26    None
27    None
28    None
29    None
30    None
31    None
32    None
33    None
34    None
35    None
36    None
37    None
38    None
39    None
40    None
41    None
42    None
43    None
44    None
45    None
46    None
47    None
48    None
49    None
50    None
51    None
52    None
53    None
54    None
55    None
56    None
57    None
58    None
Name: path, dtype: object

In [ ]:
groups = {
    "grupo_1": ["Adult 24 Média Mista", "Banknote 4 Pequena Numérica", "KC2 21 Média Numérica", "Titanic 26 Média Mista"],
    "grupo_2": ["Australian 14 Média Mista", "Breast Cancer Coimbra 9 Pequena Numérica", "Liver disorders 6 Pequena Numérica", "Room Occupancy 5 Pequena Numérica"],
    "grupo_3": ["Compas 11 Pequena Mista", "Ionosphere 33 Grande Numérica", "Vertebral Column 6 Pequena Numérica", "Give Me Some Credit 10 Pequena Numérica"],
    "grupo_4": ["Breast Cancer Wisconsin 29 Grande Numérica", "Diabetes 8 Pequena Numérica", "Phishing Websites 30 Grande Categórica", "Horse colic 27 Grande Categórica"],
    "grupo_5": ["Churn 40 Grande Categórica", "Heart 13 Média Mista", "Mammographic masses 5 Pequena Categórica", "Wine 11 Média Numérica"],
    "grupo_6": ["Default of credit card clients 23 Média Numérica", "Hill_Valley 100 Grande Numérica", "Mobile Price Classification 20 Média Numérica", "Monk1 6 Pequena Categórica"],
    "grupo_7": ["EEG Eye State 14 Média Numérica", "German 20 Média Mista", "Monk2 6 Pequena Categórica", "Mofn-3-7-10 10 Pequena Categórica"],
    "grupo_8": ["HELOC 23 Média Numérica", "Mushrooms 138 Grande Categórica", "QSAR biodegradation 41 Grande Numérica", "Sonar 60 Grande Numérica"],
    "grupo_9": ["Monk3 6 Pequena Categórica", "MUSK 166 Grande Numérica", "Spambase 57 Grande Numérica", "Student Performance - MAT 43 Grande Mista"],
    "grupo_10": ["Online news popularity 58 Grande Numérica", "Tokyo1 42 Grande Numérica", "Twonorm 20 Média Numérica", "Votes 16 Média Categórica"]
}
